In [ ]:
!pip install seaborn pandas matplotlib skunk svglib

In [ ]:
import symd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pickle
import pandas as pd
import skunk
import svglib
import seaborn as sns

In [ ]:
base_colors = [
    "f94144",
    "f3722c",
    "f8961e",
    "f9844a",
    "f9c74f",
    "90be6d",
    "43aa8b",
    "4d908e",
    "577590",
    "277da1",
]
colors = ["#" + c for c in base_colors]
sns.set_style("white")
sns.set_style("ticks")
sns.set(
    rc={
        "axes.facecolor": "#f5f4e9",
        "grid.color": "#AAAAAA",
        "axes.edgecolor": "#333333",
        "figure.facecolor": "#FFFFFF",
        "axes.grid": False,
        "axes.prop_cycle": plt.cycler("color", plt.cm.Dark2.colors),
        "font.family": "monospace",
    }
)
my_cmap = "rainbow"
print(symd.__version__)

In [ ]:
print("0\u2264x\u22641/2;0\u2264y\u22641")

In [ ]:
rot = 7
genpos = []
for ri in range(rot):
    c = np.round(np.cos(ri * np.pi * 2 / rot), 4)
    s = np.round(np.sin(ri * np.pi * 2 / rot), 4)
    if ri == 1:
        # asymm = f'0≤x≤{c};0≤y≤{s}'
        asymm = f"1/2≤x≤1.0;1/2≤y≤1.0"
    genpos.append(
        f"{c}x {-s:+}y {-0.5 * c + 0.5 * s + 0.5:+},{s}x {c:+}y {-0.5 * s - 0.5 * c + 0.5:+}"
    )
specpos = [{"name": "a", "size": 1, "sites": ["1/2, 1/2"]}]

my_group = {
    "lattice": "Oblique",
    "genpos": genpos,
    "asymm_unit": asymm,
    "specpos": specpos,
}
print(my_group)

In [ ]:
def run_sim(
    n, number_density, group, images, w=None, retries=50, pos_frames=0, steps=10**6
):
    for i in range(retries):
        try:
            np.random.seed(i)
            cell = symd.groups.get_cell(number_density, group, 2, n, w)
            md = symd.Symd(
                nparticles=n,
                cell=cell,
                ndims=2,
                images=images,
                force="lj",
                wyckoffs=w,
                group=group,
                steps=steps,
                exeDir="quasi",
                start_temperature=0.5,
                temperature=0.1,
                pressure=0.25,
            )
            md.remove_overlap()
            md.runParams["box_update_period"] = 10
            md.runParams["langevin_gamma"] = 0.5
            md.log_positions(frames=pos_frames)
            try:
                md.run()
            except RuntimeError as e:
                d = md.number_density()
                if d < 0.6:
                    print("Not dense enough, retrying", d)
                    continue

            # Basically E-min
            md.runParams["start_temperature"] = 0.1
            md.runParams["temperature"] = 1e-2
            md.runParams["langevin_gamma"] = 0.5
            md.runParams["Pressure"] = None
            md.runParams["box_update_period"] = 0
            md.runParams["steps"] = steps // 10
            if pos_frames > 0:
                md.log_positions(filename="equil.xyz", frames=pos_frames // 10)
            try:
                md.run()
            except RuntimeError as e:
                continue
            config = md.positions[-1]
            break
        except RuntimeError as e:
            print(e)
            md = None
    return md

In [ ]:
md = run_sim(256, 0.1, my_group, [0, 0], pos_frames=1000, w=[1])

In [ ]:
plt.plot(md.positions[-1, :, 0], md.positions[-1, :, 1], ".")

In [ ]:
# make a movie
import moviepy.editor as editor
from moviepy.video.io.bindings import mplfig_to_npimage


def plot_traj(
    traj, title="@_172135352171_", color="#333333", fps=60, M=0, cmap=my_cmap
):
    T, N, D = traj.shape
    fps = fps
    duration = T / fps + 2
    dpi = 90
    fig, ax = plt.subplots(figsize=(1200 / dpi, 1080 / dpi), dpi=dpi)
    tc = color  # np.linalg.norm(traj[-1] - traj[-1,M-1], axis=-1)
    points = [
        ax.scatter(
            traj[0, :, 0],
            traj[0, :, 1],
            cmap=cmap,
            c=tc,
            marker="o",
            linewidths=1,
            s=15**2,
            edgecolors="#999",
        )
    ]
    ax.set_facecolor("#f5f4e9")
    fig.patch.set_facecolor("#f5f4e9")
    title = ax.set_title(title, fontsize=32, color="#333333", fontname="monospace")
    ax.axis("off")
    xlim = np.min(traj[-1, :, 0]), np.max(traj[-1, :, 0])
    ylim = np.min(traj[-1, :, 1]), np.max(traj[-1, :, 1])
    dx = xlim[1] - xlim[0]
    dy = ylim[1] - ylim[0]
    cx, cy = traj[-1, M - 1, :2]
    s = 0.75
    xlim = [cx - dx * s, cx + dx * s]
    ylim = [cy - dy * s, cy + dy * s]
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_aspect("equal")
    # plt.tight_layout()
    def make_frame(t):
        i = int(t * fps)
        i = min(i, T - 1)
        points[0].remove()
        points[0] = ax.scatter(
            traj[i, :, 0],
            traj[i, :, 1],
            cmap=cmap,
            c=tc,
            marker="o",
            linewidths=0,
            s=7**2,
            edgecolors="#999",
        )
        plt.draw()
        return mplfig_to_npimage(fig)

    return editor.VideoClip(make_frame, duration=duration)


def write_video(clips, output, fps=60, transition=0.25):
    composite = editor.concatenate(
        clips[:1] + [c.crossfadein(transition) for c in clips[1:]],
        padding=-transition,
        method="compose",
    )
    # composite.write_videofile(output, fps=fps, preset='slower', ffmpeg_params=['-tune', 'animation', '-crf', '18'])
    composite.write_gif(output, fps=60)

In [ ]:
c = plot_traj(md.positions, M=256)
write_video([c], "7fold.gif")

In [ ]:
md.positions.shape